In [1]:
import os
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import paddle

paddle.set_device('gpu')
paddle.set_default_dtype("float64")

In [2]:
def unpickle(file):
    import pickle

    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict


def getitem(dict, index):
    label = a[b"labels"][index]
    image_data = a[b"data"][index]
    red_channel = image_data[:1024].reshape((32, 32))
    green_channel = image_data[1024:2048].reshape((32, 32))
    blue_channel = image_data[2048:].reshape((32, 32))
    rgb_image = np.dstack((red_channel, green_channel, blue_channel))
    return rgb_image, label


label_names = unpickle("../data/cifar-10-batches-py/batches.meta")[b"label_names"]

In [3]:
class Reader(paddle.io.Dataset):
    def __init__(self, *path):
        super().__init__()
        data = unpickle(path[0])
        self.x = data[b"data"] / 255
        self.y = data[b"labels"]
        for i in range(1, len(path)):
            data = unpickle(path[i])
            self.x = np.concatenate((self.x, data[b"data"] / 255))
            self.y = np.concatenate((self.y, data[b"labels"]))

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.y)

In [4]:
class MyDNN(paddle.nn.Layer):
    def __init__(self):
        super(MyDNN, self).__init__()
        self.fc1 = paddle.nn.Linear(3072, 1024)
        self.relu1 = paddle.nn.ReLU()
        self.fc2 = paddle.nn.Linear(1024, 512)
        self.relu2 = paddle.nn.ReLU()
        self.fc3 = paddle.nn.Linear(512, 256)
        self.relu3 = paddle.nn.ReLU()
        self.fc4 = paddle.nn.Linear(256, 128)
        self.relu4 = paddle.nn.ReLU()
        self.fc5 = paddle.nn.Linear(128, 64)
        self.relu5 = paddle.nn.ReLU()
        self.fc6 = paddle.nn.Linear(64, 32)
        self.relu6 = paddle.nn.ReLU()
        self.fc7 = paddle.nn.Linear(32, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        x = self.relu4(x)
        x = self.fc5(x)
        x = self.relu5(x)
        x = self.fc6(x)
        x = self.relu6(x)
        x = self.fc7(x)
        return x

In [5]:
model = MyDNN()
cross_entropy = paddle.nn.CrossEntropyLoss()
opt = paddle.optimizer.SGD(0.01, model.parameters())

In [ ]:
model.train()
epoch = 10
batch = 10
train_loader = paddle.io.DataLoader(
    Reader(
        "../data/cifar-10-batches-py/data_batch_1",
        "../data/cifar-10-batches-py/data_batch_2",
        "../data/cifar-10-batches-py/data_batch_3",
        "../data/cifar-10-batches-py/data_batch_4",
        "../data/cifar-10-batches-py/data_batch_5",
    ),
    batch_size=batch,
    shuffle=True,
)

for e in range(epoch):
    full_loss = 0
    n_loss = 0
    n_eval = 0
    correct = 0
    for i, (x, y) in enumerate(train_loader):
        if i % 100 == 0:
            print("epoch", e, "\tbatch", i, end=" ")

        y_pred = model(x)
        loss = cross_entropy(y_pred, y)

        loss.backward()
        opt.step()

        full_loss += loss.numpy()
        n_loss += batch
        opt.clear_grad()

        mask = np.array(y_pred, copy=False).argmax(1) == y
        co = mask.sum().numpy()
        correct += co

        if i % 100 == 0:
            print(f"\tloss {loss.numpy():.2f} \taccuracy {co/batch:.2f}")
    print(f"Train Accuracy: {correct/n_loss}%")
    print(f"Epoch: {e}, Loss: {full_loss / n_loss}")

epoch 0 	batch 0 	loss 2.28 	accuracy 0.10
epoch 0 	batch 100 	loss 2.27 	accuracy 0.20
epoch 0 	batch 200 	loss 2.26 	accuracy 0.00
epoch 0 	batch 300 	loss 1.87 	accuracy 0.40
epoch 0 	batch 400 	loss 1.85 	accuracy 0.30
epoch 0 	batch 500 	loss 2.28 	accuracy 0.20
epoch 0 	batch 600 	loss 1.95 	accuracy 0.30
epoch 0 	batch 700 	loss 1.91 	accuracy 0.30
epoch 0 	batch 800 	loss 1.83 	accuracy 0.20
epoch 0 	batch 900 	loss 1.91 	accuracy 0.20
epoch 0 	batch 1000 	loss 2.17 	accuracy 0.40
epoch 0 	batch 1100 	loss 2.26 	accuracy 0.10
epoch 0 	batch 1200 	loss 1.86 	accuracy 0.30
epoch 0 	batch 1300 	loss 2.08 	accuracy 0.30
epoch 0 	batch 1400 	loss 1.70 	accuracy 0.30
epoch 0 	batch 1500 	loss 1.75 	accuracy 0.40
epoch 0 	batch 1600 	loss 2.06 	accuracy 0.10
epoch 0 	batch 1700 	loss 1.95 	accuracy 0.50
epoch 0 	batch 1800 	loss 2.15 	accuracy 0.20
epoch 0 	batch 1900 	loss 2.39 	accuracy 0.10
epoch 0 	batch 2000 	loss 1.91 	accuracy 0.40
epoch 0 	batch 2100 	loss 1.47 	accuracy 0.60


In [ ]:
paddle.save(model.state_dict(), "train2.model")

In [ ]:
m = MyDNN()
m.set_state_dict(paddle.load("train2.model"))
m.eval()
eval_loader = paddle.io.DataLoader(
    Reader("../data/cifar-10-batches-py/test_batch"), batch_size=1000, shuffle=True
)

accs = []
for i, (x, y) in enumerate(eval_loader):
    y_pred = m(x)
    mask = np.array(y_pred, copy=False).argmax(1) == y
    acc = mask.sum().numpy() / batch
    print(i, acc)
    accs.append(acc)
print(np.mean(accs))

In [ ]:
a=unpickle("../data/cifar-10-batches-py/test_batch")

In [ ]:
for t in range(100):
    (x,y)=getitem(a,t)
    # plt.imshow(x)
    # plt.axis("off")
    # plt.show()
    y_pred=np.array(m(paddle.Tensor(a[b'data'][t]/255))).argmax()
    print(y==y_pred,"\t",label_names[y],"\tpred:",label_names[y_pred])

In [ ]:
m(paddle.Tensor(a[b'data'][1]/255))